In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
data = pd.read_csv('clean_data.csv')
data.head()

C:\Users\andri\AppData\Local\Temp\ipykernel_2552\2019095219.py:1: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('clean_data.csv')


,Origin,Destination,TTT,LOS,Arrival Time,Return Arrival Time,Onward Departure Airport,Onward Arrival Airport,Return Departure Airport,Return Arrival Airport,...,Departure Time Category Num,Return Time Category Num,Is Weekend Flight,Onward Departure Airport Distance,Onward Arrival Airport Distance,Return Departure Airport Distance,Return Arrival Airport Distance,Total Connections,Total Flight Duration(min),Flight Duration Binned
0,London,Paris,1,1,3/7/2025 16:30,3/8/2025 19:25,SEN,CDG,ORY,LGW,...,2,3,True,64,25,13,45,0,135,0-200
1,London,Paris,1,1,3/7/2025 16:30,3/8/2025 14:25,SEN,CDG,ORY,LHR,...,2,2,True,64,25,13,23,0,145,0-200
2,London,Paris,1,1,3/7/2025 16:30,3/8/2025 14:50,SEN,CDG,CDG,LGW,...,2,2,True,64,25,25,45,0,135,0-200
3,London,Paris,1,1,3/7/2025 16:30,3/8/2025 12:00,SEN,CDG,ORY,LGW,...,2,1,True,64,25,13,45,0,140,0-200
4,London,Paris,1,1,3/7/2025 16:30,3/8/2025 15:20,SEN,CDG,ORY,LGW,...,2,2,True,64,25,13,45,0,140,0-200


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Select important features
important_features = [
    "Total Onward Flight Duration(min)",
    "Total Return Flight Duration(min)",
    "Carry-ons",
    "Departure Day",
    "Onward Departure Airport Distance",
    "Return Day",
    "Onward Arrival Airport Distance",
    "Return Arrival Airport Distance",
    "TTT"
]

X = data[important_features]
y = data["Price(NIS)"]  # Assuming "Price" is the target variable

# Model parameters
rf_params = {
    "n_estimators": 100,
    "max_depth": None,
    "min_samples_split": 10,
    "random_state": 101
}

# 1. Train-Test Split (70-30 Random)
X_train_random, X_test_random, y_train_random, y_test_random = train_test_split(
    X, y, test_size=0.3, random_state=101
)

rf_random = RandomForestRegressor(**rf_params)
rf_random.fit(X_train_random, y_train_random)
y_pred_random = rf_random.predict(X_test_random)

# Evaluate Random Split Model
print("Random 70-30 Split:")
print("MAE:", mean_absolute_error(y_test_random, y_pred_random))
print("RMSE:", np.sqrt(mean_squared_error(y_test_random, y_pred_random)))
print("R² Score:", r2_score(y_test_random, y_pred_random))

# 2. Train-Test Split Based on TTT Condition
train_data = data[data["TTT"] <= 25]
test_data = data[data["TTT"] > 25]

X_train_TTT, y_train_TTT = train_data[important_features], train_data["Price(NIS)"]
X_test_TTT, y_test_TTT = test_data[important_features], test_data["Price(NIS)"]

rf_TTT = RandomForestRegressor(**rf_params)
rf_TTT.fit(X_train_TTT, y_train_TTT)
y_pred_TTT = rf_TTT.predict(X_test_TTT)

# Evaluate TTT-Based Model
print("\nTTT-Based Split (Train: TTT < 25, Test: TTT > 25):")
print("MAE:", mean_absolute_error(y_test_TTT, y_pred_TTT))
print("RMSE:", np.sqrt(mean_squared_error(y_test_TTT, y_pred_TTT)))
print("R² Score:", r2_score(y_test_TTT, y_pred_TTT))


Random 70-30 Split:
MAE: 57.851457392490325
RMSE: 80.37598984113222
R² Score: 0.8494780716820531

TTT-Based Split (Train: TTT < 25, Test: TTT > 25):
MAE: 108.55061907358908
RMSE: 144.6116264160397
R² Score: 0.3761139492486443
